In [ ]:
from __future__ import print_function, division   # Python 2/3 compatibility
from skimage import io, img_as_ubyte              # utilities to read and write images in various formats
import numpy as np                                # array manipulation package
import matplotlib.pylab as plt                    # plotting package
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 16)         # set default figure size
plt.rcParams['image.cmap'] = 'gray'               # set default colormap to gray

# Digital Image Processing - Programming Assignment 

The following progamming assignment involves two tasks, viz.: basic histogram processing and spatial domain image filtering tasks, i.e., image sharpening. The deadline for returning your work is **28 March 2022 at 23:59. Please, follow carefully the submission instructions given in the end of this notebook.** You are encouraged to seek information in other places than the course book and lecture material but remember **list all your sources under references**.

If you experience problems that you cannot solve using the course material or the Python documentation, or have any questions regarding to the programming assignments, please do not hesitate to contact the course assistant by e-mail at the address dip@unioulu.oulu.fi.

# 1. Histogram operations

In the following, you will have to analyze two images, `coffee.jpg` and `pout.tif`, and their histograms, and to compare the results of two histogram operations, namely histogram equalization and stretching. Now, perform the following operations in the reserved code cells and answer to the questions written in **bold** into the reserved spaces in **Finnish or English**.

**1.1. Display the images `coffee.jpg` and `pout.tif` and their histograms in the same figure.**

Hint: You can plot the histogram of an image with matplotlib's __[`hist()`](https://matplotlib.org/devdocs/api/_as_gen/matplotlib.pyplot.hist.html)__ function but please note that you have to ravel the pixels of the 2D image into 1D array first.

In [ ]:
# read the test images
coffee = (io.imread('coffee.jpg', as_gray=True) )  
coffee = img_as_ubyte(coffee)
pout = io.imread('pout.tif')

# display the images and their histograms in the same figure
fig, ax = plt.subplots(figsize=(16,16), nrows=2, ncols=2)
ax[0,0].imshow(coffee)
ax[0,0].axis('off')
ax[0,1].hist(coffee.ravel())

ax[1,0].imshow(pout)
ax[1,0].axis('off')
ax[1,1].hist(pout.ravel())

fig.tight_layout()

### Histogram equalization

**1.2. Perform histogram equalization with the function __[`exposure.equalize_hist()`](http://scikit-image.org/docs/dev/api/skimage.exposure.html#skimage.exposure.equalize_hist)__ and display the resulting images and their histograms in the same figure.**

Hint: Please note that `exposure.equalize_hist()` function returns `float64` image. You need to __[convert the image back to `uint8`](http://scikit-image.org/docs/dev/user_guide/data_types.html)__ after histogram equalization so that the intensity value range of the resulting and original histograms are comparable.

In [ ]:
from skimage import exposure
from skimage import img_as_ubyte

# perform histogram equalization and convert data type from 'float64' back to 'uint8' after histogram equalization
coffee_histeq = img_as_ubyte(exposure.equalize_hist(coffee))
pout_histeq = img_as_ubyte(exposure.equalize_hist(pout))

# display resulting images and their histograms in the same figure
fig, ax = plt.subplots(figsize=(16,16), nrows=2, ncols=2)
ax[0,0].imshow(coffee_histeq)
ax[0,0].axis('off')
ax[0,1].hist(coffee_histeq.ravel())

ax[1,0].imshow(pout_histeq)
ax[1,0].axis('off')
ax[1,1].hist(pout_histeq.ravel())

fig.tight_layout()


**Again, compare the two images. Did histogram equalization help in increasing image contrast? Why or why not?**

`In some sense the coffee image might be off worse now since all of the intensity spectrum was already represented, and now the whole image is brightened up, making the overall brightness of the image higher and making the details between dark and bright spots harder to see. The contrast in the image of the child is just slightly better since all of the spectrum is represented now, but the overall brightness is really high again and doesn't bring the best contrast.

It seems as if the problem here is that histogram equalization (by its nature) doesn't retain the shape or structure of the histogram, and as such the images are quite plain, however brighter than before.`

### Contrast stretching

Another way of improving the contrast in an image is to simply stretch the original pixel values over an extended dynamic range using a linear scaling function. For instance, in case of an `uint8` image, the desired value range of a contrast-stretched image could be the full range from 0 to 255. 

Intuitively, one could perform contrast-stretching by selecting the minimum and maximum values of the original image and map these values to 0 and 255, respectively, and linearly scale all other pixel values in between accordingly. However, even a single outlier pixel value (high or low) can affect the input scaling range too much when outcome of the histogram stretching is not particularly good.

A more robust approach is to map the intensity values so that e.g. 1st and 99th percentiles of the histogram are saturated at the minimum and maximum values of the desired intensity range. In other words, 1% of the pixels of both low and high intensities will be mapped to 0 and 255 in the contrast-stretched image while rest are scaled linearly in between.

**1.3. Now, perform contrast stretching on the original images with the help of functions __[`np.percentile()`](https://numpy.org/doc/stable/reference/generated/numpy.percentile.html)__ and __[`exposure.rescale_intensity()`](http://scikit-image.org/docs/dev/api/skimage.exposure.html#skimage.exposure.rescale_intensity)__ so that the full range from 0 and 255 is utilized based on the 1st and 99th percentiles of their histograms. Then, display the resulting images and their histograms in the same figure.**

In [ ]:
# find the 1st and 99th percentiles of each image
coffee_first = np.percentile(coffee, 1)
coffee_last = np.percentile(coffee, 99)
pout_first = np.percentile(pout, 1)
pout_last = np.percentile(pout, 99)

# rescale the intensities of both images to full 'uint8' range [0, 255] based on their 1st and 99th percentiles
coffee_rescale = exposure.rescale_intensity(coffee, in_range = (coffee_first, coffee_last), out_range = (0, 255))
pout_rescale = exposure.rescale_intensity(pout, in_range = (pout_first, pout_last), out_range = (0, 255))

# display resulting images and their histograms
fig, ax = plt.subplots(figsize=(16,16), nrows=2, ncols=2)
ax[0,0].imshow(coffee_rescale)
ax[0,0].axis('off')
ax[0,1].hist(coffee_rescale.ravel())

ax[1,0].imshow(pout_rescale)
ax[1,0].axis('off')
ax[1,1].hist(pout_rescale.ravel())

fig.tight_layout()


### Comparison

**1.4. Finally, display the original `coffee.jpg` image, and its histogram-equalized and contrast-stretched versions and the corresponding histograms into one figure (in total six images in one figure). Do the same for `pout.tif` as well.**

In [ ]:
# 6 subplots for 'coffee.jpg'
fig, ax = plt.subplots(figsize=(16,16), nrows=3, ncols=2)
ax[0,0].imshow(coffee)
ax[0,0].axis('off')
ax[0,1].hist(coffee.ravel())

ax[1,0].imshow(coffee_histeq)
ax[1,0].axis('off')
ax[1,1].hist(coffee_histeq.ravel())

ax[2,0].imshow(coffee_rescale)
ax[2,0].axis('off')
ax[2,1].hist(coffee_rescale.ravel())

fig.tight_layout()


# 6 subplots for 'pout.tif'
fig, ax = plt.subplots(figsize=(16,16), nrows=3, ncols=2)
ax[0,0].imshow(pout)
ax[0,0].axis('off')
ax[0,1].hist(pout.ravel())

ax[1,0].imshow(pout_histeq)
ax[1,0].axis('off')
ax[1,1].hist(pout_histeq.ravel())

ax[2,0].imshow(pout_rescale)
ax[2,0].axis('off')
ax[2,1].hist(pout_rescale.ravel())

fig.tight_layout()


# 2. Image sharpening

First, read the part concerning sharpening spatial transforms in the lecture notes or in the course book.

In this exercise, your task is to perform a sharpening transform to the image `moonunsharp.tif` in spatial domain enhancing the details, like edges, in the original grayscale image. The use of built-in functions that perform image sharpening from scratch, like `scipy.misc.imfilter()`, is forbidden but functions like __[`scipy.signal.convolve2d()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve2d.html)__ can be used for the task. You can select some method presented in the lecture notes or the course book, e.g. use Laplacian operator and convolution, for sharpening the test image. 

Please note that it does not matter what method you use or how “good” the sharpening looks as long as the sharpening can be observed in the end result. An example result achieved with __[`ImageFilter`](https://pillow.readthedocs.io/en/stable/reference/ImageFilter.html)__ is shown below:

In [ ]:
# image sharpening example using 'ImageFilter' module from PILLOW with 'image.filter()' function
from PIL import ImageFilter, Image

moon = Image.open('moonunsharp.tif')
moon_sharp_example = moon.filter(ImageFilter.SHARPEN)

fig, ax = plt.subplots(1, 2)
ax[0].imshow(moon, vmin=0, vmax=255, cmap=plt.get_cmap('gray'))
ax[0].set_title('original')
ax[0].axis('off')
ax[1].imshow(moon_sharp_example, vmin=0, vmax=255, cmap=plt.get_cmap('gray'))
ax[1].set_title('sharpened')
ax[1].axis('off')
fig.tight_layout()

**2.1. Now, implement your own image sharpening transform and apply it on the test image.**

Hint: Like in the previous task, please note the __[image data type (`dtype`) and corresponding value range](http://scikit-image.org/docs/dev/user_guide/data_types.html)__ after filtering/sharpening as unexpected errors with arithmetic may occur (see pre-tutorials) !

In [ ]:
# perform image sharpening using e.g. a Laplacian mask and convolution
from scipy import signal

# defining Laplacian mask
mask_laplace = [[0, -1, 0], [-1, 4, -1], [0, -1, 0]]

# sharpening the image with convolution
laplace_conv = img_as_ubyte(signal.convolve2d(moon, mask_laplace, mode = 'same'))
sharp_laplace = moon + (0.91 * laplace_conv)



**2.2. Display the original and sharpened moon images in the same figure.**

In [ ]:
# plot original and sharpened moon images in the same figure
fig, ax = plt.subplots(1, 2)
ax[0].imshow(moon, vmin=0, vmax=255, cmap=plt.get_cmap('gray'))
ax[0].set_title('original')
ax[0].axis('off')
ax[1].imshow(sharp_laplace, vmin=0, vmax=255, cmap=plt.get_cmap('gray'))
ax[1].set_title('sharpened')
ax[1].axis('off')
fig.tight_layout()

# Aftermath
Finally, fill your answers to the following questions:

# References
`Laplacian filter from course material.`

# Submission

1. Before submitting your work, **check that your notebook (code) runs from scratch** and reproduces all the requested results by clicking on the menu `Kernel -> Restart & Run All`! Also, check that you have answered all the questions written in **bold**.
2. Clear all outputs and variables, etc. by click on the menu `Kernel -> Restart & Clear Output`. This may (or will) reduce the file size of your deliverable a lot! 
3. Rename this Jupyter notebook to **`DIP_PA1_[student number(s)].ipynb`** (e.g. `DIP_PA1_1234567.ipynb` if solo work or `DIP_PA1_1234567-7654321.ipynb` if working in a pair)